In [141]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import plotly
from config import user, apikey
plotly.tools.set_credentials_file(username=user, api_key=apikey)
import plotly.plotly as py
import plotly.figure_factory as ff

In [146]:
country_codes = pd.read_csv("country_codes.csv", encoding='ISO-8859-1')

In [147]:
url = "http://dancecal.com/?sMon=1&sYear=2016&num=24&theme=&hidedanceIcon=&list=1&hidetype="
response = requests.get(url)

soup = bs(response.text, 'html.parser')
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">

<html>
<head>
<title>DanceCal.com -  Exchange Calendar -  Workshops Parties Events</title>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="/Themes/css/DanceCal.com.css" rel="stylesheet" type="text/css"/>
<script src="/Themes/js/jquery.js"></script>
<script src="/Themes/js/DanceCal.com.js"></script>
<link href="http://DanceCal.com/favicon.gif" rel="SHORTCUT ICON"/>
</head>
<body>
<div class="DCHeader">
<div class="DCLogoFull">
<div class="DCLogo">
<a href="http://DanceCal.com/" name="top">
<h1>DanceCal.com</h1></a>
</div>
<div class="DCTagline">
<a href="http://DanceCal.com/">
<h2>Dance Event and Exchange Calendar</h2></a>
</div>
<div class="DCSubtagline">
<h3>Your resource for local and regional dance exchanges, workshops and other events.</h3>
</div>
</div>
<div class="DCMap">
<center>
<a href="?map">
<img align="right" 

In [148]:
lindy_list = []
for x in soup.find_all('div', class_="DCListEvent"):
    try:
        name = x.find('span', class_="DCEventInfoName").text
        genre = x.find('span', class_="DCEventInfoDances").text[8:].split(", ")
        date = x.find('span', class_="DCEventInfoDate").text
        
        if len(x.find('span', class_="DCEventInfoWhere").text[6:].split(", ")) == 1:
            location = x.find('span', class_="DCEventInfoWhere").text[6:].split(", ")[0]
        elif len(x.find('span', class_="DCEventInfoWhere").text[6:].split(", ")) == 2:
            location = x.find('span', class_="DCEventInfoWhere").text[6:].split(", ")[1]
        elif len(x.find('span', class_="DCEventInfoWhere").text[6:].split(", ")) == 3:
            location = x.find('span', class_="DCEventInfoWhere").text[6:].split(", ")[2]
        elif len(x.find('span', class_="DCEventInfoWhere").text[6:].split(", ")) == 4:
            location = x.find('span', class_="DCEventInfoWhere").text[6:].split(", ")[3]
        else:
            location = "error"


        lindy_list.append({
            "Name": name,
            "Genre": genre,
            "Date": date,
            "Location": location
        })
    except AttributeError as e:
        print(e)
lindy_list

'NoneType' object has no attribute 'text'


[{'Name': 'Moscow Christmas Swing Dance Camp',
  'Genre': ['Lindy', 'Balboa'],
  'Date': '1/3/2016',
  'Location': 'Russia'},
 {'Name': 'Krakow Lindy Invasion',
  'Genre': ['Lindy', 'Solo Jazz'],
  'Date': '1/8/2016',
  'Location': 'Poland'},
 {'Name': 'California Balboa Classic 2016',
  'Genre': ['Balboa'],
  'Date': '1/15/2016',
  'Location': 'USA'},
 {'Name': 'City In Motion Dance Weekend',
  'Genre': ['Fusion'],
  'Date': '1/15/2016',
  'Location': 'USA'},
 {'Name': 'DecoDance at Art Deco Weekend',
  'Genre': ['Lindy'],
  'Date': '1/15/2016',
  'Location': 'USA'},
 {'Name': 'LiLi Hop',
  'Genre': ['Lindy', 'Solo Jazz'],
  'Date': '1/15/2016',
  'Location': 'Belgium'},
 {'Name': 'My Lindy Kraze',
  'Genre': ['Lindy', 'Shag'],
  'Date': '1/15/2016',
  'Location': 'USA'},
 {'Name': 'Anderson Blues & Lindy Experience',
  'Genre': ['Blues', 'Lindy', 'Shag'],
  'Date': '1/22/2016',
  'Location': 'USA'},
 {'Name': 'Freedom Swing',
  'Genre': ['West Coast'],
  'Date': '1/22/2016',
  'Locat

In [151]:
lindy_df = pd.DataFrame(lindy_list)
lindy_df.head()

Date               Genre Location                               Name
0   1/3/2016     [Lindy, Balboa]   Russia  Moscow Christmas Swing Dance Camp
1   1/8/2016  [Lindy, Solo Jazz]   Poland              Krakow Lindy Invasion
2  1/15/2016            [Balboa]      USA     California Balboa Classic 2016
3  1/15/2016            [Fusion]      USA       City In Motion Dance Weekend
4  1/15/2016             [Lindy]      USA      DecoDance at Art Deco Weekend

In [152]:
lindy_codes = pd.merge(lindy_df, country_codes, on="Location", how="left")
lindy_codes.head()

Date               Genre Location                               Name  \
0   1/3/2016     [Lindy, Balboa]   Russia  Moscow Christmas Swing Dance Camp   
1   1/8/2016  [Lindy, Solo Jazz]   Poland              Krakow Lindy Invasion   
2  1/15/2016            [Balboa]      USA     California Balboa Classic 2016   
3  1/15/2016            [Fusion]      USA       City In Motion Dance Weekend   
4  1/15/2016             [Lindy]      USA      DecoDance at Art Deco Weekend   

  Code  
0  NaN  
1  POL  
2  NaN  
3  NaN  
4  NaN

In [153]:
# BAD PRACTICE, CHANGE!!!
for index, row in lindy_codes.iterrows():
#     if row["Location"] == "USA":
#         row["Code"] = "USA"
    if row["Location"] == "Russia":
        row["Code"] = "RUS"

In [154]:
test = lindy_codes.groupby(['Code']).count().reset_index()
test[["Code", "Name"]]

Code  Name
0   ARG     3
1   AUS    26
2   AUT     6
3   BEL     6
4   BLR     2
5   BRA     1
6   CAN    28
7   CHE    10
8   CHL     1
9   CHN     1
10  CUB     1
11  CZE     3
12  DEU    39
13  DNK     6
14  ESP    27
15  EST     3
16  FRA    29
17  GBR    43
18  GRC     1
19  HKG     1
20  HRV     1
21  HUN     1
22  IRL     3
23  ISL     1
24  ISR     2
25  ITA    13
26  JAM     1
27  JPN     1
28  LTU     4
29  MEX     3
30  MOZ     1
31  MYS     2
32  NLD     9
33  NOR     3
34  NZL     5
35  POL    13
36  PRI     1
37  PRT     4
38  RUS     6
39  SGP     2
40  SVN     4
41  SWE    23
42  THA     1
43  TUR     1
44  UKR     2
45  ZAF     2

In [155]:
data = [ dict(
        type = 'choropleth',
        locations = test['Code'],
        z = test['Name'],
        text = test['Code'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = 'Lindy Events'),
      ) ]

layout = dict(
    title = '2014 Global GDP<br>Source:\
            <a href="https://www.cia.gov/library/publications/the-world-factbook/fields/2195.html">\
            CIA World Factbook</a>',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )